# 7-round Speck (FNN)


## Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from pipeline import *

2023-05-15 15:44:36.042853: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from dataset_rr.make_train_data import make_train_data
from dataset_rr.speck import Speck

## Importing the dataset

In [75]:
n_train_samples = 10**6
n_eval_samples = 10**5
n_rounds = 7

cipher = Speck(n_rounds=n_rounds)

key = cipher.draw_keys(1)

In [76]:
train_samples, train_labels = make_train_data(n_train_samples, cipher, key)
test_samples, test_labels = make_train_data(n_eval_samples, cipher, key)

In [77]:
get_dataset_info(train_labels, train_samples, test_labels, test_samples)

===== Training Labels Shape: (1000000, 32)
===== Label Shape: (32,)
===== Training Samples Shape: (1000000, 32)
===== Sample Shape: (32,)
===== Testing Labels Shape: (100000, 32)
===== Testing Samples Shape: (100000, 32)


## Creating the model

In [78]:
# Imports
from keras import Sequential
from keras.layers import Input, Dense, BatchNormalization, LayerNormalization
from keras.optimizers import Adam

### Model hyperparameters
In this code block, we specify most parameters and hyperparameters that will be used in the training of the neural network.

Add customization here.

In [88]:
input_shape = np.shape(train_samples[0])

# output dimension
dim = len(train_labels[0])

# units per hidden layer
units = dim*32

loss_scc = 'sparse_categorical_crossentropy'
loss_mse = 'mse'
loss_bce = 'binary_crossentropy'
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=1000,
    decay_rate=0.01)
learning_rate = 0.1

optimizer = Adam(learning_rate=0.001)
metrics = ['accuracy', 'binary_accuracy']
epochs = 2
batch_size = 5000

### Model
In this code block, we create the model, according to the parameters and the topology we want to achieve. 
We then compile it specifying the optimizer, the loss and the metrics we want outputted.

Add customization here.

In [89]:
# Type of model
neural_network = Sequential()

# Input layer
neural_network.add(Input(shape=input_shape))

# Hidden layers
#neural_network.add(BatchNormalization())
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))

# Output layer
neural_network.add(Dense(units=dim, activation='sigmoid'))

# Summary
neural_network.summary()

# Compile model
neural_network.compile(optimizer=optimizer, loss=loss_mse, metrics=metrics)

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_67 (Dense)            (None, 1024)              33792     
                                                                 
 dense_68 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_69 (Dense)            (None, 32)                32800     
                                                                 
Total params: 1,116,192
Trainable params: 1,116,192
Non-trainable params: 0
_________________________________________________________________


### Training
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

This block mostly stays the same.

In [90]:
history = train_model(neural_network, train_samples, train_labels, 
                      batch_size=batch_size, 
                      epochs=epochs)

Epoch 1/2
180/180 [==============================] - 27s 147ms/step - loss: 0.2500 - accuracy: 0.0404 - binary_accuracy: 0.5000 - val_loss: 0.2500 - val_accuracy: 0.0475 - val_binary_accuracy: 0.5001
Epoch 2/2
180/180 [==============================] - 28s 158ms/step - loss: 0.2500 - accuracy: 0.0416 - binary_accuracy: 0.5017 - val_loss: 0.2500 - val_accuracy: 0.0211 - val_binary_accuracy: 0.4999


In [91]:
h_loss = history.history['loss']
h_val_loss = history.history['val_loss']
h_val_accuracy = history.history['val_accuracy']

print(h_loss)
print(h_val_loss)
print(h_val_accuracy)

[0.2500428259372711, 0.24999727308750153]
[0.2500045597553253, 0.2500157654285431]
[0.04751000180840492, 0.021080000326037407]


### Testing
Here, we evaluate the neural network with the test data.

This block stays the same.

In [92]:
results = neural_network.evaluate(test_samples, test_labels, batch_size=batch_size)
print("Test loss: {}".format(results[0]))
print("Test accuracy: {}".format(results[1]))

20/20 [==============================] - 1s 49ms/step - loss: 0.2500 - accuracy: 0.0212 - binary_accuracy: 0.5000
Test loss: 0.25001680850982666
Test accuracy: 0.021209999918937683
